In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
salaries = pd.read_excel('salaries_1985to2020_final.xlsx')
p_stats = pd.read_excel('Complete_Train_Set_With_Cap_Usage.xlsx')
stats_2020 = pd.read_excel('Clean_2019_2020_Data.xlsx')

In [4]:
salary_file = pd.ExcelFile('salaries_1985to2020_final.xlsx')
players_info = pd.read_excel(salary_file, 1)

In [5]:
salary_caps = pd.read_excel('salary_caps.xlsx')

In [6]:
salaries = salaries.rename(columns={'names':'name'})
stats_2020 = stats_2020.merge(salaries.loc[salaries['season_end']==2020], on=['name'])
salary_cap_2020 = salary_caps[salary_caps['season']=='2019-20']['salary_cap']
individual_salaries = stats_2020['salary']
cap_usage = individual_salaries/int(salary_cap_2020)
stats_2020['cap_usage'] = cap_usage

In [7]:
years = list(salaries.groupby('player_id').apply(lambda x: [i for i in range(1, len(x)+1)]))
flatten = lambda l: [item for sublist in l for item in sublist]
salaries['years'] = flatten(years)

In [8]:
salaries.head()

,name,player_id,salary,season,season_end,season_start,team,key,years
0,Alaa Abdelnaby,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers,Alaa Abdelnaby1990-91,1
1,Alaa Abdelnaby,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers,Alaa Abdelnaby1991-92,2
2,Alaa Abdelnaby,abdelal01,500000,1992-93,1993,1992,Boston Celtics,Alaa Abdelnaby1992-93,3
3,Alaa Abdelnaby,abdelal01,805000,1993-94,1994,1993,Boston Celtics,Alaa Abdelnaby1993-94,4
4,Alaa Abdelnaby,abdelal01,650000,1994-95,1995,1994,Sacramento Kings,Alaa Abdelnaby1994-95,5


In [9]:
salaries.loc[salaries['name']=='LeBron James']

,name,player_id,salary,season,season_end,season_start,team,key,years
6787,LeBron James,jamesle01,4018920,2003-04,2004,2003,Cleveland Cavaliers,LeBron James2003-04,1
6788,LeBron James,jamesle01,4320360,2004-05,2005,2004,Cleveland Cavaliers,LeBron James2004-05,2
6789,LeBron James,jamesle01,4621800,2005-06,2006,2005,Cleveland Cavaliers,LeBron James2005-06,3
6790,LeBron James,jamesle01,5828090,2006-07,2007,2006,Cleveland Cavaliers,LeBron James2006-07,4
6791,LeBron James,jamesle01,13041250,2007-08,2008,2007,Cleveland Cavaliers,LeBron James2007-08,5
6792,LeBron James,jamesle01,14410581,2008-09,2009,2008,Cleveland Cavaliers,LeBron James2008-09,6
6793,LeBron James,jamesle01,15779912,2009-10,2010,2009,Cleveland Cavaliers,LeBron James2009-10,7
6794,LeBron James,jamesle01,14500000,2010-11,2011,2010,Miami Heat,LeBron James2010-11,8
6795,LeBron James,jamesle01,16022500,2011-12,2012,2011,Miami Heat,LeBron James2011-12,9
6796,LeBron James,jamesle01,17545000,2012-13,2013,2012,Miami Heat,LeBron James2012-13,10


In [10]:
p_stats.head()

,Unnamed: 0,name,positions,age,team_x,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,...,season,player_id,salary,season_end,season_start,team_y,key,salary_cap_x,salary_cap_y,cap_usage
0,0,Kareem Abdul-Jabbar,CENTER,37,LOS ANGELES LAKERS,79,1.000000,33.291139,9.151899,15.278481,...,1984-85,abdulka01,1530000,1985,1984,Los Angeles Lakers,Kareem Abdul-Jabbar1984-85,3600000,3600000,0.425000
1,1,Alvan Adams,POWER FORWARD,30,PHOENIX SUNS,82,0.841463,26.048780,5.804878,11.158537,...,1984-85,adamsal01,500000,1985,1984,Phoenix Suns,Alvan Adams1984-85,3600000,3600000,0.138889
2,2,Mark Aguirre,SMALL FORWARD,25,DALLAS MAVERICKS,80,0.987500,33.737500,9.925000,19.612500,...,1984-85,aguirma01,800000,1985,1984,Dallas Mavericks,Mark Aguirre1984-85,3600000,3600000,0.222222
3,3,Danny Ainge,SHOOTING GUARD,25,BOSTON CELTICS,75,0.973333,34.186667,5.586667,10.560000,...,1984-85,aingeda01,400000,1985,1984,Boston Celtics,Danny Ainge1984-85,3600000,3600000,0.111111
4,4,John Bagley,POINT GUARD,24,CLEVELAND CAVALIERS,81,0.802469,29.641975,4.172840,8.555556,...,1984-85,baglejo01,209000,1985,1984,Cleveland Cavaliers,John Bagley1984-85,3600000,3600000,0.058056


In [11]:
p_stats = p_stats.merge(salaries, on = ['name', 'season'])
p_stats.head()

,Unnamed: 0,name,positions,age,team_x,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,...,salary_cap_x,salary_cap_y,cap_usage,player_id_y,salary_y,season_end_y,season_start_y,team,key_y,years
0,0,Kareem Abdul-Jabbar,CENTER,37,LOS ANGELES LAKERS,79,1.000000,33.291139,9.151899,15.278481,...,3600000,3600000,0.425000,abdulka01,1530000,1985,1984,Los Angeles Lakers,Kareem Abdul-Jabbar1984-85,1
1,1,Alvan Adams,POWER FORWARD,30,PHOENIX SUNS,82,0.841463,26.048780,5.804878,11.158537,...,3600000,3600000,0.138889,adamsal01,500000,1985,1984,Phoenix Suns,Alvan Adams1984-85,1
2,2,Mark Aguirre,SMALL FORWARD,25,DALLAS MAVERICKS,80,0.987500,33.737500,9.925000,19.612500,...,3600000,3600000,0.222222,aguirma01,800000,1985,1984,Dallas Mavericks,Mark Aguirre1984-85,1
3,3,Danny Ainge,SHOOTING GUARD,25,BOSTON CELTICS,75,0.973333,34.186667,5.586667,10.560000,...,3600000,3600000,0.111111,aingeda01,400000,1985,1984,Boston Celtics,Danny Ainge1984-85,1
4,4,John Bagley,POINT GUARD,24,CLEVELAND CAVALIERS,81,0.802469,29.641975,4.172840,8.555556,...,3600000,3600000,0.058056,baglejo01,209000,1985,1984,Cleveland Cavaliers,John Bagley1984-85,1


In [12]:
stats_2020 = stats_2020.merge(salaries, on = ['name', 'season'])
stats_2020.head()

,Unnamed: 0,name,positions,age,team_x,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,...,team_y,key_x,cap_usage,player_id_y,salary_y,season_end_y,season_start_y,team,key_y,years
0,0,Steven Adams,C,26,OKC,63,63,26.7,4.5,7.6,...,Oklahoma City Thunder,Steven Adams2019-20,0.236785,adamsst01,25842697,2020,2019,Oklahoma City Thunder,Steven Adams2019-20,7
1,1,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,11.0,...,Miami Heat,Bam Adebayo2019-20,0.031648,adebaba01,3454080,2020,2019,Miami Heat,Bam Adebayo2019-20,3
2,2,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,15.0,...,San Antonio Kings,LaMarcus Aldridge2019-20,0.238226,aldrila01,26000000,2020,2019,San Antonio Kings,LaMarcus Aldridge2019-20,14
3,4,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,5.7,...,New Orleans Pelicans,Nickeil Alexander-Walker2019-20,0.027165,alexani01,2964840,2020,2019,New Orleans Pelicans,Nickeil Alexander-Walker2019-20,1
4,5,Grayson Allen,SG,24,MEM,38,0,18.9,3.1,6.6,...,Memphis Grizzlies,Grayson Allen2019-20,0.022259,allengr01,2429400,2020,2019,Memphis Grizzlies,Grayson Allen2019-20,2


In [13]:
p_stats.columns

Index(['Unnamed: 0', 'name', 'positions', 'age', 'team_x', 'games_played',
       'games_started', 'minutes_played', 'made_field_goals',
       'attempted_field_goals', 'made_three_point_field_goals',
       'attempted_three_point_field_goals', 'made_free_throws',
       'attempted_free_throws', 'offensive_rebounds', 'defensive_rebounds',
       'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls', 'points',
       'player_efficiency_rating', 'true_shooting_percentage',
       'three_point_attempt_rate', 'free_throw_attempt_rate',
       'offensive_rebound_percentage', 'defensive_rebound_percentage',
       'total_rebound_percentage', 'assist_percentage', 'steal_percentage',
       'block_percentage', 'turnover_percentage', 'usage_percentage',
       'offensive_win_shares', 'defensive_win_shares', 'win_shares',
       'win_shares_per_48_minutes', 'offensive_box_plus_minus',
       'defensive_box_plus_minus', 'box_plus_minus',
       'value_over_replacement_player', 'season', 'p

In [14]:
p_stats_v2 = p_stats.drop(columns=['team_y', 'key_x', 'player_id_y', 'salary_y', 'season_end_y',
       'season_start_y', 'team', 'key_y'])
stats_2020_v2 = stats_2020.drop(columns=['team_y', 'key_x', 'player_id_y', 'salary_y', 'season_end_y',
       'season_start_y', 'team', 'key_y'])

# model starts from here

,feature,r_squared_value
29,turnover_percentage,0.000007
48,years,0.001091
43,salary_cap_y,0.002544
42,salary_cap_x,0.002544
0,Unnamed: 0,0.003553
41,season_start_x,0.004937
47,season_start_y,0.004937
46,season_end_y,0.004937
40,season_end_x,0.004937
27,steal_percentage,0.009707


In [16]:
def extract_numerical_data(df):
    columns_to_remove = []
    for feature in df.columns.values:
        if type(df.iloc[0][feature])==str:
            columns_to_remove.append(feature)
    return df.drop(columns=columns_to_remove)

def clean_datasets(p_stats, stats_2020, filter_year):
    p_stats = p_stats.dropna()
    stats_2020 = stats_2020.dropna()
    print(p_stats.shape, stats_2020.shape, ' before any filters')
    p_stats = p_stats.loc[p_stats['season_end_x'] >= filter_year]
    stats_2020 = stats_2020.loc[stats_2020['season_end_x'] >= filter_year]
    
    print(p_stats.shape, stats_2020.shape, ' after year filter')
    p_stats = p_stats.loc[p_stats['games_played']>= 35]
    stats_2020 = stats_2020.loc[stats_2020['games_played']>= 35]
    
    print(p_stats.shape, stats_2020.shape, ' after games_played filter')
    p_stats = p_stats.loc[p_stats['cap_usage'] >= 0.01]
    stats_2020 = stats_2020.loc[stats_2020['cap_usage'] >= 0.01]
    print(p_stats.shape, stats_2020.shape, ' after g_league filter')
    
    #add filter on years-played only for p_stats to take out rookies
    p_stats = p_stats.loc[p_stats['years'] > 4]
    print(p_stats.shape, ' p_stats after years_played filter to take out rookies')
    stats_2020_wo_rookies = stats_2020.loc[stats_2020['years'] > 4]
    
    X_train = extract_numerical_data(p_stats)
    X_test = extract_numerical_data(stats_2020)
    X_train = X_train.drop(columns=['Unnamed: 0'])
    X_test = X_test.drop(columns=['Unnamed: 0'])
    X_train = X_train.dropna()
    X_test = X_test.dropna()
    X_train = X_train.loc[X_train['season_end_x'] >= filter_year]
    X_test = X_test.loc[X_test['season_end_x'] >= filter_year]
    y_train = X_train[['cap_usage']]
    y_test = X_test[['cap_usage']]
    X_train = X_train.drop(columns=['cap_usage'])
    X_test = X_test.drop(columns=['cap_usage'])
    X_train = X_train.drop(columns=['salary_cap_x', 'salary_cap_y'])
    print("is the cleaning is successful?:", X_train.columns.values==X_test.columns.values)
    return X_train, X_test, y_train, y_test, p_stats, stats_2020, stats_2020_wo_rookies



In [18]:
X_train, X_test, y_train, y_test, p_stats, stats_2020, wo_rook_2020 = clean_datasets(p_stats, stats_2020, 2015)

(16083, 59) (829, 58)  before any filters
(3260, 59) (829, 58)  after year filter
(1719, 59) (402, 58)  after games_played filter
(1615, 59) (345, 58)  after g_league filter
(989, 59)  p_stats after years_played filter to take out rookies
is the cleaning is successful?: [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True]


In [20]:
def identify_features_linear(X, y):
    rankings_df = pd.DataFrame(columns=['feature', 'r_squared_value'])
    for feature in X.columns.values:
        model = LinearRegression().fit(X[[feature]], y)
        rankings_df = rankings_df.append({'feature': feature, 'r_squared_value': model.score(X[[feature]], y)}, ignore_index=True)
    return rankings_df.sort_values(by=['r_squared_value'])
identify_features_linear(extract_numerical_data(p_stats), y_train)

,feature,r_squared_value
29,turnover_percentage,0.000007
48,years,0.001091
43,salary_cap_y,0.002544
42,salary_cap_x,0.002544
0,Unnamed: 0,0.003553
41,season_start_x,0.004937
47,season_start_y,0.004937
46,season_end_y,0.004937
40,season_end_x,0.004937
27,steal_percentage,0.009707


In [21]:
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge

In [22]:
features = ['points', 'made_field_goals', 'minutes_played', 'defensive_rebounds', 'win_shares', 'value_over_replacement_player', 'turnovers']
X_train_new = X_train[features]
X_test_new = X_test[features]

In [23]:
def model_performance(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    print("score for the training set: ", model.score(X_train, y_train))
    print("score for the test set: ", model.score(X_test, y_test))
    print("mean squared error for the training set: ", mean_squared_error(model.predict(X_train), y_train))
    print("mean squared error for the test set: ", mean_squared_error(model.predict(X_test), y_test))

## model building starts here

#### model 1: boosting

In [24]:
import sklearn.ensemble as ske

In [25]:
from sklearn.ensemble import AdaBoostRegressor

In [26]:
model = AdaBoostRegressor(n_estimators=5)
model_performance(model, X_train_new, y_train, X_test_new, y_test)

score for the training set:  0.5618640123686759
score for the test set:  0.21627975282649992
mean squared error for the training set:  0.0030701549663885884
mean squared error for the test set:  0.0052104823629439715


C:\Users\risha\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [28]:
train_with_name = pd.concat([p_stats[['name']], X_train_new], axis=1)
test_with_name = pd.concat([stats_2020[['name']], X_test_new], axis=1)

In [29]:
def make_dataframe_of_results(test_df, y_test, model, features):
    #pass in a fitted model
    prediction = pd.DataFrame(columns=['name', 'prediction', 'actual'])
    prediction['name'] = test_df['name']
    prediction['prediction'] = model.predict(test_df[features])
    prediction['actual'] = y_test
    prediction['error'] = prediction['prediction'] - prediction['actual']
    prediction['error_squared'] = prediction['error']*prediction['error']
    return prediction

In [30]:
prediction1 = make_dataframe_of_results(test_with_name, y_test, model, features)
prediction1.loc[prediction1['name']=='Mike Conley']

,name,prediction,actual,error,error_squared
195,Mike Conley,0.099329,0.297889,-0.19856,0.039426


In [31]:
model_linear = LinearRegression()
model_linear.fit(train_with_name[features], y_train)
prediction_linear = make_dataframe_of_results(test_with_name, y_test, model_linear, features)
prediction_linear.loc[prediction_linear['name']=='Mike Conley']

,name,prediction,actual,error,error_squared
195,Mike Conley,0.125032,0.297889,-0.172858,0.02988


#### model 2: random forest regressor

In [32]:
model_rf = ske.RandomForestRegressor(max_depth = 5, min_samples_split=3, min_samples_leaf=1)
model_rf.fit(train_with_name[features], y_train)
prediction_rf = make_dataframe_of_results(test_with_name, y_test, model_rf, features)
model_performance(model_rf, train_with_name[features], y_train, test_with_name[features], y_test)

<ipython-input-32-4ad1f91bae2c>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf.fit(train_with_name[features], y_train)
<ipython-input-23-c491b32f82f9>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


score for the training set:  0.6919818722788336
score for the test set:  0.2371465266478736
mean squared error for the training set:  0.0021583787026337492
mean squared error for the test set:  0.005071751792488595


In [33]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [ ]:
#attempt1 - bad
pipe = Pipeline([('regressor', ske.RandomForestRegressor())])
samples_split = np.arange(2, 5)
samples_leaf = np.arange(1, 4)

params = [{'regressor__min_samples_split': samples_split, 'regressor__min_samples_leaf':samples_leaf, }]

gridsearch = GridSearchCV(pipe, params, verbose=1).fit(train_with_name[features], np.ravel(y_train))
print('The best score is: ', gridsearch.best_score_)
print('The best parameters are: ', gridsearch.best_params_)

In [ ]:
#attempt2 - better
pipe = Pipeline([('regressor', ske.RandomForestRegressor())])
samples_split = np.arange(2, 5)
samples_leaf = np.arange(1, 4)
                 
params = [{'regressor__min_samples_split': [3], 'regressor__min_samples_leaf': [3], 'regressor__max_features':np.arange(1, 8, 2), 'regressor__max_depth':np.arange(3, 6), 'regressor__n_estimators': np.arange(10, 200, 40)}]

gridsearch = GridSearchCV(pipe, params, verbose=1).fit(train_with_name[features], np.ravel(y_train))
print('The best score is: ', gridsearch.best_score_)
print('The best parameters are: ', gridsearch.best_params_)

In [34]:
model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
prediction_rf = make_dataframe_of_results(test_with_name, y_test, model_rf, features)
model_performance(model_rf, train_with_name[features], y_train, test_with_name[features], y_test)

<ipython-input-34-e6916b4a9474>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
<ipython-input-23-c491b32f82f9>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


score for the training set:  0.6322487671090832
score for the test set:  0.30351158800345246
mean squared error for the training set:  0.0025769471258444833
mean squared error for the test set:  0.004630530600416487


In [ ]:
pd.set_option('display.max_rows', None)
prediction_rf.sort_values(by= ['error_squared'])

In [35]:
#test on 2020 without rookies
model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
prediction_rf = make_dataframe_of_results(wo_rook_2020[features + ['name']], wo_rook_2020['cap_usage'], model_rf, features)
model_performance(model_rf, train_with_name[features], y_train, wo_rook_2020[features], wo_rook_2020['cap_usage'])

<ipython-input-35-de8b1338c387>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
<ipython-input-23-c491b32f82f9>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


score for the training set:  0.6336387800617374
score for the test set:  0.57804583822284
mean squared error for the training set:  0.0025672068732963942
mean squared error for the test set:  0.0034273657739517876


In [ ]:
prediction_rf.sort_values(by = ['error_squared'])

In [ ]:
wo_rook_2020.loc[wo_rook_2020['name']=='John Collins']

## taking out max star players

In [ ]:
p_stats_noMAX = p_stats.loc[p_stats['cap_usage'] <= 0.2]
stats_2020_noMAX = stats_2020.loc[stats_2020['cap_usage'] <= 0.2]
X_train_noMAX, X_test_noMAX, y_train_noMAX, y_test_noMAX, p_stats_noMAX_v2, stats_2020_noMAX, wo_rook_2020_noMAX = clean_datasets(p_stats_noMAX, stats_2020_noMAX, 2015)

In [ ]:
features = ['points', 'made_field_goals', 'minutes_played', 'defensive_rebounds', 'win_shares', 'value_over_replacement_player', 'turnovers']
X_train_noMAX = X_train_noMAX[features]
X_test_noMAX = X_test_noMAX[features]

In [ ]:
train_with_name_noMAX = pd.concat([p_stats_noMAX_v2[['name']], X_train_noMAX], axis=1)
test_with_name_noMAX = pd.concat([stats_2020_noMAX[['name']], X_test_noMAX], axis=1)

In [ ]:
#test on 2020 without rookies and without max players
model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name_noMAX[features], y_train_noMAX)
prediction_rf = make_dataframe_of_results(wo_rook_2020[features + ['name']], wo_rook_2020['cap_usage'], model_rf, features)
model_performance(model_rf, train_with_name_noMAX[features], y_train_noMAX, wo_rook_2020[features], wo_rook_2020['cap_usage'])

In [ ]:
prediction_rf.sort_values(by = ['error_squared'])

# FINAL MODELS

## 1. Random Forest Regressor

In [36]:
#test on 2020 data without rookies in the training set or the test set ==> signifcantly better results
model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
prediction_rf = make_dataframe_of_results(wo_rook_2020[features + ['name']], wo_rook_2020['cap_usage'], model_rf, features)
model_performance(model_rf, train_with_name[features], y_train, wo_rook_2020[features], wo_rook_2020['cap_usage'])

<ipython-input-36-5ce155582b10>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
<ipython-input-23-c491b32f82f9>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


score for the training set:  0.6365315487140946
score for the test set:  0.5775592215667622
mean squared error for the training set:  0.0025469363447496243
mean squared error for the test set:  0.0034313183674397904


## 2. Bagging Regressor with Random Forest

In [90]:
model_bag = ske.BaggingRegressor(base_estimator=model_rf, n_estimators=50, random_state=0, max_samples=1.0,max_features = 1.0, bootstrap_features=False, verbose=0.5).fit(train_with_name[features], y_train)
prediction_bag = make_dataframe_of_results(wo_rook_2020[features + ['name']], wo_rook_2020['cap_usage'], model_bag, features)
model_performance(model_bag, train_with_name[features], y_train, wo_rook_2020[features], wo_rook_2020['cap_usage'])

C:\Users\risha\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
C:\Users\risha\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.5s finished
[Parallel(n_jobs=1)]: Using backend Sequent

score for the training set:  0.6189138398935979


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


score for the test set:  0.5808540876467547


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


mean squared error for the training set:  0.0026703891031593027
mean squared error for the test set:  0.0034045554811946138


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


## 3. Extremely Randomized Trees

In [111]:
model_ert = ske.ExtraTreesRegressor(max_depth = 10, max_features = 7, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 175).fit(train_with_name[features], y_train)
prediction_ert = make_dataframe_of_results(wo_rook_2020[features + ['name']], wo_rook_2020['cap_usage'], model_ert, features)
model_performance(model_ert, train_with_name[features], y_train, wo_rook_2020[features], wo_rook_2020['cap_usage'])

<ipython-input-111-69bca508f681>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_ert = ske.ExtraTreesRegressor(max_depth = 10, max_features = 7, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 175).fit(train_with_name[features], y_train)
<ipython-input-23-c491b32f82f9>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


score for the training set:  0.803302995332376
score for the test set:  0.6001714805133411
mean squared error for the training set:  0.001378317012986884
mean squared error for the test set:  0.003247647984716633


In [148]:
model_boost_ert = AdaBoostRegressor(base_estimator=model_ert, n_estimators=10, random_state = 1).fit(train_with_name[features], y_train)
prediction_boost_ert = make_dataframe_of_results(wo_rook_2020[features + ['name']], wo_rook_2020['cap_usage'], model_boost_ert, features)
model_performance(model_boost_ert, train_with_name[features], y_train, wo_rook_2020[features], wo_rook_2020['cap_usage'])

C:\Users\risha\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\risha\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


score for the training set:  0.8440204879448775
score for the test set:  0.5777000686204865
mean squared error for the training set:  0.0010929968939092597
mean squared error for the test set:  0.0034301743228609583


In [146]:
model_gbr = ske.GradientBoostingRegressor(n_estimators = 35).fit(train_with_name[features], y_train)
prediction_gbr = make_dataframe_of_results(wo_rook_2020[features + ['name']], wo_rook_2020['cap_usage'], model_gbr, features)
model_performance(model_gbr, train_with_name[features], y_train, wo_rook_2020[features], wo_rook_2020['cap_usage'])

score for the training set:  0.6660566653561149
score for the test set:  0.5559458730875537
mean squared error for the training set:  0.002340044680858328
mean squared error for the test set:  0.003606874997871256


C:\Users\risha\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\risha\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


## 4. Voting Regressor based on previous models

In [151]:
model_vote = ske.VotingRegressor([('ert', model_ert), ('bag', model_bag), ('rf', model_rf)], weights=[2, 0.5, 1]).fit(train_with_name[features], y_train)
prediction_vote = make_dataframe_of_results(wo_rook_2020[features + ['name']], wo_rook_2020['cap_usage'], model_vote, features)
model_performance(model_vote, train_with_name[features], y_train, wo_rook_2020[features], wo_rook_2020['cap_usage'])

C:\Users\risha\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
C:\Users\risha\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.1s finished
[Parallel(n_jobs=1)]: Using backend Sequent

score for the training set:  0.737908416873845


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


score for the test set:  0.5966290974003243


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


mean squared error for the training set:  0.00183655713819269
mean squared error for the test set:  0.0032764213533418973


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
